In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


***
날씨와 농산물 가격은 어떤 상관관계가 있는지 예측한다.  
기상 정보 -> 기상자료개방포털, 농산물 가격 정보 -> 카미스 검색; 농산물 유통 정보

기상 정보 : 평균기온 (avgTemp), 최저기온 (minTemp),최고기온(maxTemp), 강수량(rainFall)
가격 정보 : 평균가격 (avgPrice)

In [2]:
# price_data.csv 파일을 읽어서 데이터프레임으로 저장한다.
# csv파일의 첫 줄에 컬럼 이름(헤더)이 없을 경우 header=None 옵션을 지정해서 읽은 후 컬럼 이름을 붙여준다.
price_data = pd.read_csv('./price_data.csv', header=None)
price_data.columns = ('year', 'avgTemp', 'minTemp', 'maxTemp', 'rainFall', 'avgPrice')
price_data

,year,avgTemp,minTemp,maxTemp,rainFall,avgPrice
0,20100101,-4.9,-11.0,0.9,0.0,2123
1,20100102,-3.1,-5.5,5.5,0.8,2123
2,20100103,-2.9,-6.9,1.4,0.0,2123
3,20100104,-1.8,-5.1,2.2,5.9,2020
4,20100105,-5.2,-8.7,-1.8,0.7,2060
...,...,...,...,...,...,...
2917,20171227,-3.9,-8.0,0.7,0.0,2865
2918,20171228,-1.5,-6.9,3.7,0.0,2884
2919,20171229,2.9,-2.1,8.0,0.0,2901
2920,20171230,2.9,-1.6,7.1,0.6,2901


In [3]:
# 데이터프레임에 저장된 데이터를 numpy 배열로 변환한다.
data = np.array(price_data)
print(data.shape)
print(data)

(2922, 6)
[[ 2.0100101e+07 -4.9000000e+00 -1.1000000e+01  9.0000000e-01
   0.0000000e+00  2.1230000e+03]
 [ 2.0100102e+07 -3.1000000e+00 -5.5000000e+00  5.5000000e+00
   8.0000000e-01  2.1230000e+03]
 [ 2.0100103e+07 -2.9000000e+00 -6.9000000e+00  1.4000000e+00
   0.0000000e+00  2.1230000e+03]
 ...
 [ 2.0171229e+07  2.9000000e+00 -2.1000000e+00  8.0000000e+00
   0.0000000e+00  2.9010000e+03]
 [ 2.0171230e+07  2.9000000e+00 -1.6000000e+00  7.1000000e+00
   6.0000000e-01  2.9010000e+03]
 [ 2.0171231e+07  2.1000000e+00 -2.0000000e+00  5.8000000e+00
   4.0000000e-01  2.9010000e+03]]


In [4]:
#numpy 배열에서 변화 요인(피쳐) 데이터(평균기온, 최저기온, 최고기온, 강수량)로 사용할 데이터를 뽑아낸다.
xData = data[:, 1:5]
print(xData)

[[ -4.9 -11.    0.9   0. ]
 [ -3.1  -5.5   5.5   0.8]
 [ -2.9  -6.9   1.4   0. ]
 ...
 [  2.9  -2.1   8.    0. ]
 [  2.9  -1.6   7.1   0.6]
 [  2.1  -2.    5.8   0.4]]


In [5]:
#numpy 배열에서 변화 요인에 따른 레이블 데이터(평균가격)로 사용할 데이터를 뽑아낸다.
#yData = data[:, -1] # 인덱싱으로 뽑아내면 1차원 형태로 데이터를 뽑아낸다.
#변화 요인이 2차원이므로 결과도 2차원 형태로 뽑아내야 한다.
#1차원 형태로 뽑아냈으면 reshape(-1, 1)을 실행해서 2차원 형태로 변형한다.
#yData = data[:, -1].reshape(-1, 1)
#yData = data[:, 5:6] #슬라이싱으로 뽑아내며 2차원 데이터를 뽑아낸다.
yData = data[:, [-1]] # 인덱싱으로 뽑아낸 결과를 리스트화 시켜도 2차원 형태의 데이터로 뽑아낸다.
print(yData)

[[2123.]
 [2123.]
 [2123.]
 ...
 [2901.]
 [2901.]
 [2901.]]


In [6]:
# 입력 데이터와 입력 데이터에 따른 레이블을 저장할 placeholder를 만든다.
X = tf.placeholder(dtype=tf.float32, shape=(None, 4))#변화 요인을 기억할 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=(None, 1))#변화 요인에 따른 결과 레이블 데이터(결과)를 기억할 placeholder

In [7]:
# 다변인 선형 회귀 모델의 기울기(가중치)와 y절편(바이어스)를 임의의 값으로 초기화한다.
a = tf.Variable(tf.random_uniform((4, 1), dtype=tf.float32))# 기울기, 입력데이터와 행렬곱하기 위해서 4 x 1 로 난수를 발생시킨다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32))# y절편;바이어스
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('기울기\n', sess.run(a), sep='')
print('y절편\n', sess.run(b), sep='')

기울기
[[0.32814336]
 [0.28359067]
 [0.90508807]
 [0.4887681 ]]
y절편
[0.07346058]


In [8]:
# 다변인 선형 회귀 모델의 가설(예측값을 계산하는 식)을 만든다.
y = tf.matmul(X, a) + b # 예측값 계산식

#평균 제곱근 오차 함수(RMSE)를 만든다. -> 예측값(y)과 실제값(Y)의 편차의 제곱에 대한 평균
loss = tf.sqrt( tf.reduce_mean( tf.square(y - Y) ) )

#경사 하강법 알고리즘을 사용해서 오차함수의 결과를 최소로 하는 기울기와 y절편을 찾는다.
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)


In [9]:
#학습시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(100001):
     _, loss_, y_ = sess.run([train, loss, y], feed_dict={ X : xData, Y: yData})
     if epoch % 5000 == 0:
          print('epoch -> {:5d}, loss -> {:9.4f}, price -> {:8.2f}'.format ( epoch, loss_, y_[0][0]))
     

epoch ->     0, loss -> 3521.2180, price ->   -11.55
epoch ->  5000, loss -> 1733.0333, price ->   813.30
epoch -> 10000, loss -> 1565.6587, price ->  1717.91
epoch -> 15000, loss -> 1521.5409, price ->  2176.65
epoch -> 20000, loss -> 1511.1726, price ->  2393.30
epoch -> 25000, loss -> 1508.4528, price ->  2493.75
epoch -> 30000, loss -> 1507.3824, price ->  2540.15
epoch -> 35000, loss -> 1506.6649, price ->  2561.60
epoch -> 40000, loss -> 1506.0245, price ->  2571.54
epoch -> 45000, loss -> 1505.4022, price ->  2576.23
epoch -> 50000, loss -> 1504.7859, price ->  2578.45
epoch -> 55000, loss -> 1504.1725, price ->  2579.69
epoch -> 60000, loss -> 1503.5609, price ->  2580.17
epoch -> 65000, loss -> 1502.9540, price ->  2580.52
epoch -> 70000, loss -> 1502.3489, price ->  2580.79
epoch -> 75000, loss -> 1501.7451, price ->  2581.05
epoch -> 80000, loss -> 1501.1434, price ->  2581.29
epoch -> 85000, loss -> 1500.5460, price ->  2581.50
epoch -> 90000, loss -> 1499.9498, price ->  2

In [10]:
# 테스트
data = [[5.2, -7.5, 700.1, 15.0]]
result = sess.run(y, feed_dict={X : data})
print(result)

[[181345.69]]


In [11]:
# 학습 완료된 예측 모델을 디스크에 저장한다.
# Saver() 객체는 학습된 모델을 디스크에 저장 또는 불러올 때 사용한다.
saver = tf.train.Saver()
saver.save(sess, './model/baechu.ckpt' )
print('학습된 모델 저장!!!!')

학습된 모델 저장!!!!


In [12]:
sess.close()